In [1]:
import pandas as pd
import numpy as np
import re
import time
import category_encoders as ce
from collections import Counter
from xgboost import XGBClassifier
from xgboost import plot_tree
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
import seaborn as sns
import matplotlib
from matplotlib import pyplot
%matplotlib inline

In [2]:
#data_path = '/kaggle/input/icr-identify-age-related-conditions/train.csv'
data_path = 'train.csv'

In [3]:
df = pd.read_csv(data_path)

In [4]:
df.head(3)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0


In [5]:
# drop 'Id' column
df.drop(['Id'], axis=1, inplace=True)

In [6]:
#dealing with Null values
#df.fillna(0,inplace=True)
#df.isnull().values.any()

In [7]:
# splitting the data
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [8]:
# encoding categorical variable
enc = ce.OneHotEncoder(cols=["EJ"])
enc.fit(X)
encoded_X = enc.transform(X)
encoded_X.head(2)

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000


In [9]:
encoded_X.shape

(617, 57)

In [10]:
# splitting the data
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(encoded_X, y, test_size = test_size, random_state = 0)

#ec1_eval_set = [(X_train, y_train), (X_test, y_test)]

In [17]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split

def objective(trial, encoded_X, y):

    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'booster': 'gbtree',
        'verbosity': 0,
        'n_jobs': -1,
        'seed': 1
    }

    # Define the search space for hyperparameters
    param_space = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.01, 10.0)
    }

    # Train the XGBoost model with the selected hyperparameters
    clf = xgb.XGBClassifier(**param, **param_space)
    clf.fit(X_train, y_train)

    # Evaluate the model on the validation set
    #y_pred = model.predict_proba(X_valid)[:, 1]
    cv_scores = cross_val_score(clf, encoded_X, y, cv = 20, scoring="f1_macro") 
    score = np.mean(cv_scores)
    return score

In [18]:
# Create and run an Optuna study

# More options for creating the optuna study can be found at their webpage:
# https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html
#
# The default sampler is called TPESampler and is very good, but there are others.

study = optuna.create_study(direction="maximize")

[I 2023-07-18 23:29:02,862] A new study created in memory with name: no-name-5714af48-1986-486e-a67d-dda0e029c313


In [19]:
study.optimize(lambda trial: objective(trial, encoded_X, y), n_trials=100,  gc_after_trial=True)

C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha

[I 2023-07-18 23:29:12,611] Trial 4 finished with value: 0.45201640464798365 and parameters: {'max_depth': 5, 'learning_rate': 0.016063091916820527, 'subsample': 0.8, 'colsample_bytree': 0.7, 'min_child_weight': 39, 'gamma': 0.06401418907543246, 'reg_alpha': 0.17427253243834856, 'reg_lambda': 0.39514744680365305}. Best is trial 2 with value: 0.7398397416801968.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial

[I 2023-07-18 23:29:19,951] Trial 9 finished with value: 0.45201640464798365 and parameters: {'max_depth': 1, 'learning_rate': 0.00559299299175247, 'subsample': 0.5, 'colsample_bytree': 1.0, 'min_child_weight': 24, 'gamma': 0.044158096871965195, 'reg_alpha': 0.7622514460612725, 'reg_lambda': 0.024915234054676712}. Best is trial 2 with value: 0.7398397416801968.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial

[I 2023-07-18 23:29:34,975] Trial 14 finished with value: 0.7406870264899441 and parameters: {'max_depth': 10, 'learning_rate': 6.705890449574089e-05, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 13, 'gamma': 0.46919583216349187, 'reg_alpha': 0.2862321217612535, 'reg_lambda': 0.03533770166604716}. Best is trial 11 with value: 0.7541516508334727.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': tri

[I 2023-07-18 23:29:43,075] Trial 19 finished with value: 0.5436166686489428 and parameters: {'max_depth': 7, 'learning_rate': 0.08630909175780134, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 31, 'gamma': 0.08144153130238908, 'reg_alpha': 1.6992958402422358, 'reg_lambda': 4.804885470558383}. Best is trial 11 with value: 0.7541516508334727.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.su

...

[I 2023-07-18 23:32:31,292] Trial 79 finished with value: 0.7455501407773092 and parameters: {'max_depth': 7, 'learning_rate': 3.652743507197386e-05, 'subsample': 0.4, 'colsample_bytree': 0.7, 'min_child_weight': 3, 'gamma': 0.09166539939915426, 'reg_alpha': 0.15263903974967805, 'reg_lambda': 0.4606171583492119}. Best is trial 56 with value: 0.8212288883579116.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial

[I 2023-07-18 23:32:44,718] Trial 84 finished with value: 0.7749830894819993 and parameters: {'max_depth': 7, 'learning_rate': 1.3091759291941594e-05, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 4, 'gamma': 0.09968415312280211, 'reg_alpha': 0.19318231639772243, 'reg_lambda': 0.5139535136383867}. Best is trial 56 with value: 0.8212288883579116.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': tria

[I 2023-07-18 23:33:00,882] Trial 89 finished with value: 0.7967446825411785 and parameters: {'max_depth': 4, 'learning_rate': 1.6233332972231305e-05, 'subsample': 0.6, 'colsample_bytree': 0.9, 'min_child_weight': 1, 'gamma': 0.07805908495260526, 'reg_alpha': 0.11619245076683175, 'reg_lambda': 1.1236557743850695}. Best is trial 56 with value: 0.8212288883579116.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': tria

[I 2023-07-18 23:33:16,278] Trial 94 finished with value: 0.8280592211654128 and parameters: {'max_depth': 7, 'learning_rate': 3.935770307293339e-05, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0.10295543042219762, 'reg_alpha': 0.18723120322324682, 'reg_lambda': 0.6128169300568804}. Best is trial 93 with value: 0.8318631427340403.
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
C:\Users\chris\AppData\Local\Temp\ipykernel_40572\1296519654.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial

[I 2023-07-18 23:33:31,855] Trial 99 finished with value: 0.7277496195997861 and parameters: {'max_depth': 7, 'learning_rate': 6.550425685489964e-05, 'subsample': 0.7, 'colsample_bytree': 0.6, 'min_child_weight': 6, 'gamma': 0.09561161515874066, 'reg_alpha': 0.11185057825476913, 'reg_lambda': 1.1750183427619179}. Best is trial 97 with value: 0.8408031788883619.


In [20]:
best_params = study.best_params
print('best_params: ', best_params)
best_score = study.best_value
print('best_score: ', best_score)

best_params:  {'max_depth': 7, 'learning_rate': 5.495568561536644e-05, 'subsample': 0.7, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0.09713551852723058, 'reg_alpha': 0.14538846198743027, 'reg_lambda': 0.9524402910037023}
best_score:  0.8408031788883619


In [21]:
# All the details of the best trial
study.best_trial

FrozenTrial(number=97, state=TrialState.COMPLETE, values=[0.8408031788883619], datetime_start=datetime.datetime(2023, 7, 18, 23, 33, 22, 541704), datetime_complete=datetime.datetime(2023, 7, 18, 23, 33, 26, 463358), params={'max_depth': 7, 'learning_rate': 5.495568561536644e-05, 'subsample': 0.7, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0.09713551852723058, 'reg_alpha': 0.14538846198743027, 'reg_lambda': 0.9524402910037023}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=10, log=False, low=1, step=1), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'subsample': CategoricalDistribution(choices=(0.4, 0.5, 0.6, 0.7, 0.8, 1.0)), 'colsample_bytree': CategoricalDistribution(choices=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'min_child_weight': IntDistribution(high=50, log=False, low=1, step=1), 'gamma': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'reg_alpha': FloatDistribut

In [22]:
# testing the model
test_df = pd.read_csv('test.csv')
test_df.head(3)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
new_test_df = test_df.drop(['Id'],axis=1)
new_test_df.head(3)

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
encoded_test_df = enc.transform(new_test_df)
encoded_test_df.head(3)

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
clf = XGBClassifier(**study.best_params)
clf.fit(encoded_X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.09713551852723058, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=5.495568561536644e-05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [26]:
#X_test_df = test_df.drop(['Id'], axis=1)

# Extract IDs from the test dataset
test_ids = test_df['Id']

# Make predictions
predictions = clf.predict_proba(encoded_test_df)

# Create a new DataFrame with prediction results
prediction_df = pd.DataFrame({'Id': test_ids, 'class_0': predictions[:, 0], 'class_1': predictions[:, 1]})

# Print the prediction DataFrame
print(prediction_df)

             Id   class_0   class_1
0  00eed32682bb  0.500758  0.499242
1  010ebe33f668  0.500758  0.499242
2  02fa521e1838  0.500758  0.499242
3  040e15f562a2  0.500758  0.499242
4  046e85c7cc7f  0.500758  0.499242


In [27]:
prediction_df.to_csv('submission.csv', index=False)